# Набор для начала работы с челленджем по машинному отучению NeurIPS 2023

[![Открыть в Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/code/wasjaip/test-finetune-v1-005)

Спасибо [Sebastian Oleszko](https://www.kaggle.com/sebastianoleszko) за предоставленные материалы.

Этот блокнот является частью набора для начала работы с [челленджем по машинному отучению NeurIPS 2023](https://unlearning-challenge.github.io/). В нем объясняется процесс работы челленджа и содержатся примеры кода для отучения и оценки.

В блокноте три раздела:

  * 💾 В первом разделе мы загрузим пример набора данных (CIFAR10) и предварительно обученную модель (ResNet18).

  * 🎯 Во втором разделе мы разработаем алгоритм отучения. Начнем с разделения исходного обучающего набора на набор для сохранения и набор для забывания. Цель алгоритма отучения - обновить предварительно обученную модель так, чтобы она максимально приблизилась к модели, обученной только на наборе для сохранения, но не на наборе для забывания. Мы предоставляем простой алгоритм отучения в качестве отправной точки для участников для разработки собственных алгоритмов отучения.

  * 🏅 В третьем разделе мы оценим наш алгоритм отучения, используя простые атаки на инференцию членства (MIA). Обратите внимание, что это другой метод оценки, чем тот, который будет использоваться при подаче заявок на соревнование.
  

Подчеркиваем, что этот блокнот предоставляется для удобства, чтобы помочь участникам быстро начать работу. Заявки будут оцениваться другим способом, чем указано в этом блокноте, на другом (приватном) наборе данных человеческих лиц. Для запуска блокнота требуется установленная актуальная версия Python и Pytorch.


In [ ]:
# Импорт стандартных библиотек и модулей
import os                   # Работа с операционной системой и файловой системой
import subprocess           # Запуск новых приложений и процессов, подключение к их каналам ввода/вывода

# Импорт библиотек для работы с данными, нейронными сетями и обучением
import pandas as pd         # Библиотека для анализа и обработки данных
import torch                # Основная библиотека для работы с тензорами и графами вычислений
import torchvision          # Библиотека с инструментами для работы с изображениями, предварительно обученными моделями и трансформациями
import torch.nn as nn       # Модуль для построения слоев нейронных сетей
import torch.optim as optim # Модуль для оптимизаторов
from torchvision.models import resnet18 # Импорт модели ResNet18
from torch.utils.data import DataLoader, Dataset # Инструменты для работы с датасетами и загрузки данных
import torch.nn.functional as F # Функциональный интерфейс для нейронных сетей
import torch.nn.utils.prune as prune # Инструменты для обрезки (прореживания) моделей
from math import sqrt                 # Импорт функции квадратного корня
import json                           # Работа с JSON-файлами
from copy import deepcopy             # Глубокое копирование структур данных

# Определение устройства для обучения (использование GPU, если доступно)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # Использовать GPU (cuda) если доступно, иначе CPU


In [ ]:
# проверка

if DEVICE != 'cuda':
    raise RuntimeError('Make sure you have added an accelerator to your notebook; the submission will fail otherwise!')

## Вспомогательные функции для загрузки скрытого датасета

In [ ]:

def load_example(df_row):
    # Загрузка изображения по пути, указанному в строке датафрейма
    image = torchvision.io.read_image(df_row['image_path'])
    # Формирование словаря с данными об изображении
    result = {
        'image': image,
        'image_id': df_row['image_id'],
        'age_group': df_row['age_group'],
        'age': df_row['age'],
        'person_id': df_row['person_id']
    }
    return result

class HiddenDataset(Dataset):
    '''Класс скрытого датасета.'''
    def __init__(self, split='train'):
        super().__init__()
        self.examples = [] # Список для хранения примеров

        # Чтение данных из CSV файла для указанного раздела (split)
        df = pd.read_csv(f'/kaggle/input/neurips-2023-machine-unlearning/{split}.csv')
        # Формирование полного пути к изображениям
        df['image_path'] = df['image_id'].apply(
            lambda x: os.path.join('/kaggle/input/neurips-2023-machine-unlearning/', 'images', x.split('-')[0], x.split('-')[1] + '.png'))
        df = df.sort_values(by='image_path') # Сортировка по пути изображения
        # Добавление каждого примера в список examples
        df.apply(lambda row: self.examples.append(load_example(row)), axis=1)
        # Проверка на наличие примеров
        if len(self.examples) == 0:
            raise ValueError('No examples.')

    def __len__(self):
        # Возвращает количество примеров в датасете
        return len(self.examples)

    def __getitem__(self, idx):
        # Получение примера по индексу
        example = self.examples[idx]
        image = example['image']
        # Преобразование типа данных изображения
        image = image.to(torch.float32)
        example['image'] = image
        return example

def get_dataset(batch_size):
    '''Функция получения датасета.'''
    # Создание объектов датасетов для разных разделов
    retain_ds = HiddenDataset(split='retain')     # Датасет для сохранения
    forget_ds = HiddenDataset(split='forget')     # Датасет для забывания
    val_ds = HiddenDataset(split='validation')    # Валидационный датасет

    # Создание DataLoader для каждого датасета
    retain_loader = DataLoader(retain_ds, batch_size=batch_size, shuffle=True)
    forget_loader = DataLoader(forget_ds, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

    return retain_loader, forget_loader, validation_loader


### функция kl_loss_fn рассчитывает потерю Kullback-Leibler Divergence, которая меряет, насколько одно вероятностное распределение отличается от другого. Это часто используется в задачах машинного обучения для сравнения распределения предсказаний модели с истинным распределением.


In [ ]:
def kl_loss_fn(outputs, dist_target):
    # Функция потерь Kullback-Leibler Divergence
    # Расчет KL-дивергенции между выходами модели и целевым распределением
    kl_loss = F.kl_div(torch.log_softmax(outputs, dim=1), dist_target, log_target=True, reduction='batchmean')
    return kl_loss


### функция entropy_loss_fn комбинирует потери кросс-энтропии и энтропии распределений. Кросс-энтропия рассчитывается между выходами модели и истинными метками, а также используется среднеквадратичное отклонение для сравнения энтропии предсказаний модели с энтропией целевого распределения. Эта комбинированная функция потерь может быть полезна для более точной настройки модели на основе различных аспектов информации, содержащейся в данных.

In [ ]:
def entropy_loss_fn(outputs, labels, dist_target, class_weights):
    # Функция потерь, комбинирующая кросс-энтропию и энтропию распределений
    # Расчет кросс-энтропии между выходами модели и истинными метками
    ce_loss = F.cross_entropy(outputs, labels, weight=class_weights)
    # Расчет энтропии для целевого распределения
    entropy_dist_target = torch.sum(-torch.exp(dist_target) * dist_target, dim=1)
    # Расчет энтропии для предсказаний модели
    entropy_outputs = torch.sum(-torch.softmax(outputs, dim=1) * torch.log_softmax(outputs, dim=1), dim=1)
    # Среднеквадратичное отклонение (MSE) между энтропиями предсказаний и целевого распределения
    entropy_loss = F.mse_loss(entropy_outputs, entropy_dist_target)
    # Комбинирование кросс-энтропии и MSE для энтропии
    return ce_loss + entropy_loss


# Новая функция отучения.

код представляет собой функцию, которая реализует процесс "отучения" нейронной сети. Он включает в себя итерации по данным, сохранение исходного состояния сети, дообучение с применением обрезки весов, и расчет потерь, с последующим обновлением параметров сети. Это позволяет модифицировать сеть таким образом, чтобы "забыть" некоторые из изначально обученных паттернов.

In [ ]:
def unlearning(
    net, 
    retain_loader, 
    forget_loader, 
    val_loader,
    class_weights=None,
):
    """Простое отучение путем дообучения."""
    epochs = 3.5  # Количество эпох для дообучения
    max_iters = int(len(retain_loader) * epochs)  # Максимальное количество итераций
    optimizer = optim.SGD(net.parameters(), lr=0.0005,
                      momentum=0.9, weight_decay=5e-4)  # Оптимизатор
    initial_net = deepcopy(net)  # Создание копии исходной сети
    
    net.train()  # Перевод сети в режим обучения
    initial_net.eval()  # Перевод исходной сети в режим оценки
    
    def prune_model(net, amount=0.95, rand_init=True):
        # Модули для прореживания
        modules = list()
        for k, m in enumerate(net.modules()):
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                modules.append((m, 'weight'))
                if m.bias is not None:
                    modules.append((m, 'bias'))

        # Критерии прореживания
        prune.global_unstructured(
            modules,
            # Метод прореживания: prune.RandomUnstructured,
            pruning_method=prune.L1Unstructured,
            amount=amount,
        )

        # Выполнение прореживания
        for k, m in enumerate(net.modules()):
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                prune.remove(m, 'weight')
                if m.bias is not None:
                    prune.remove(m, 'bias')

        # Случайная инициализация
        if rand_init:
            for k, m in enumerate(net.modules()):
                # Для сверточных слоев (Conv2d)
                if isinstance(m, nn.Conv2d):
                    mask = m.weight == 0  # Создание маски для обнуленных весов
                    c_in = mask.shape[1]  # Количество входных каналов
                    # Коэффициент для инициализации на основе размера фильтра и количества входных каналов
                    k = 1/(c_in*mask.shape[2]*mask.shape[3])
                    # Генерация случайных весов в диапазоне [-sqrt(k), sqrt(k)]
                    randinit = (torch.rand_like(m.weight)-0.5)*2*sqrt(k)
                    m.weight.data[mask] = randinit[mask]  # Применение случайных весов только к обнуленным элементам

                # Для полносвязных слоев (Linear)
                if isinstance(m, nn.Linear):
                    mask = m.weight == 0  # Создание маски для обнуленных весов
                    c_in = mask.shape[1]  # Количество входных единиц
                    # Коэффициент для инициализации, основанный на количестве входных единиц
                    k = 1/c_in
                    # Генерация случайных весов в диапазоне [-sqrt(k), sqrt(k)]
                    randinit = (torch.rand_like(m.weight)-0.5)*2*sqrt(k)
                    m.weight.data[mask] = randinit[mask]  # Применение случайных весов только к обнуленным элементам
    
    num_iters = 0  # Счетчик итераций
    running = True  # Переменная для контроля цикла
    prune_amount = 0.995  # Степень прореживания
    prune_model(net, prune_amount, True)  # Прореживание модели
    while running:
        net.train()  # Перевод сети в режим обучения
        for sample in retain_loader:
            inputs = sample["image"]  # Входные данные
            targets = sample["age_group"]  # Целевые метки
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            # Получение целевого распределения
            with torch.no_grad():
                original_outputs = initial_net(inputs)
                preds = torch.log_softmax(original_outputs, dim=1)
            
            optimizer.zero_grad()  # Обнуление градиентов
            outputs = net(inputs)  # Получение предсказаний от обучаемой сети
            loss = entropy_loss_fn(outputs, targets, preds, class_weights)  # Расчет потерь
            loss.backward()  # Расчет градиентов
            optimizer.step()  # Обновление весов модели

            num_iters += 1  # Увеличение счетчика итераций
            # Остановка при достижении максимального количества итераций
            if num_iters > max_iters:
                running = False
                break
        
    net.eval()  # Перевод сети в режим оценки


## submission

In [ ]:
# Проверка наличия файла, указывающего на "mock submission" (тестовую подачу)
if os.path.exists('/kaggle/input/neurips-2023-machine-unlearning/empty.txt'):
    # Создание и модификация модели для тестовой подачи
    net = resnet18(weights=None, num_classes=10)  # Инициализация модели ResNet18 без предварительного обучения
    # Проход по всем модулям модели и применение обрезки (pruning) к сверточным и полносвязным слоям
    for k, m in enumerate(net.modules()):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            prune.l1_unstructured(m, name="weight", amount=0.95)  # Прореживание весов на 95%
            prune.remove(m, 'weight')  # Удаление обрезанных весов

    print(m)
    subprocess.run('touch submission.zip', shell=True)  # Создание пустого файла submission.zip
else:
    # Обработка реальных данных
    # Важно: создание контрольных точек вне рабочего каталога, чтобы избежать нехватки дискового пространства
    class_weights_fname = "/kaggle/input/neurips-2023-machine-unlearning/age_class_weights.json"
    with open(class_weights_fname) as f:
        class_weights = json.load(f)  # Загрузка весов классов из JSON-файла

    # Удаление лишних уровней в словаре, если они есть
    while isinstance(class_weights, dict):
        if len(class_weights) > 1:
            class_weights = list(class_weights.values())
            break
        for _, class_weights in class_weights.items():
            break

    # Преобразование списка весов в тензор
    class_weights = torch.tensor(class_weights).to(DEVICE, dtype=torch.float32)
    # Коррекция весов для учета дисбаланса в данных
    class_weights = class_weights ** -0.1

    os.makedirs('/kaggle/tmp', exist_ok=True)  # Создание временной директории
    retain_loader, forget_loader, validation_loader = get_dataset(64)  # Получение загрузчиков данных
    net = resnet18(weights=None, num_classes=10)  # Создание модели ResNet18
    net.to(DEVICE)  # Перемещение модели на выбранное устройство (GPU/CPU)

    # Процесс обучения/отучения
    for i in range(512):
        # Загрузка исходного состояния модели
        net.load_state_dict(torch.load('/kaggle/input/neurips-2023-machine-unlearning/original_model.pth'))
        # Выполнение процедуры отучения
        unlearning(net, retain_loader, forget_loader, validation_loader, class_weights=class_weights)
        # Сохранение контрольной точки после отучения
        net = net.to(torch.half)
        state = net.state_dict()
        torch.save(state, f'/kaggle/tmp/unlearned_checkpoint_{i}.pth')
        net = net.to(torch.float)
        
    # Проверка наличия ровно 512 контрольных точек
    unlearned_ckpts = os.listdir('/kaggle/tmp')
    if len(unlearned_ckpts) != 512:
        raise RuntimeError('Ожидалось ровно 512 контрольных точек. В противном случае подача вызовет исключение.')


    # Создание архива с контрольными точками для подачи
    subprocess.run('zip submission.zip /kaggle/tmp/*.pth', shell=True)
